Displaying full slicer Viewer in separate tab

In [1]:
import JupyterNotebooksLib as slicernb
import slicer
slicer.mrmlScene.Clear(False)
# Load a Slicer sample data set
import SampleData


# Set viewer size to 50% of screen size
slicernb.AppWindow.setWindowSize(scale=0.5)
# Hide patient information from slice view
slicernb.showSliceViewAnnotations(False)

#slicernb.showVolumeRendering(volume, show=True)
app = slicernb.AppWindow()
app
app.setContents("full")
slicer.util.selectModule("Data")
from ipywidgets import HTML
HTML(f"""<a href="{slicernb.AppWindow.defaultDesktopUrl()}" target="_blank">
<b>Click here</b> to open application window in a new browser tab.</a>""")

HTML(value='<a href="/desktop/" target="_blank">\n<b>Click here</b> to open application window in a new browse…

In [2]:

#numbEnd=43;32; 40
numbEnd=19
# volume = slicer.util.loadVolume(f'/home/sliceruser/data/100{numbEnd}/100{numbEnd}_10000{numbEnd}_adc.mha')
# volume = slicer.util.loadVolume(f'/home/sliceruser/data/100{numbEnd}/100{numbEnd}_10000{numbEnd}_cor.mha')
# volume = slicer.util.loadVolume(f'/home/sliceruser/data/100{numbEnd}/100{numbEnd}_10000{numbEnd}_hbv.mha')
# volume = slicer.util.loadVolume(f'/home/sliceruser/data/100{numbEnd}/100{numbEnd}_10000{numbEnd}_sag.mha')
volume = slicer.util.loadVolume(f'/home/sliceruser/data/100{numbEnd}/100{numbEnd}_10000{numbEnd}_t2w.mha')
#originalLabel= slicer.util.loadSegmentation(f'/home/sliceruser/labels/csPCa_lesion_delineations/human_expert/original/100{numbEnd}_10000{numbEnd}.nii.gz')
resampledLabel= slicer.util.loadSegmentation(f'/home/sliceruser/labels/csPCa_lesion_delineations/human_expert/resampled/100{numbEnd}_10000{numbEnd}.nii.gz')

In [ ]:
#show from itk 
#https://slicer.readthedocs.io/en/latest/developer_guide/script_repository.html?highlight=sitk#running-an-itk-filter-in-python-using-simpleitk

In [3]:

imagee


In [11]:
import JupyterNotebooksLib as slicernb
import sitkUtils
import slicer
import pandas as pd
import torch
import pandas as pd
import numpy as np
import torchio as tio
from torch.utils.data import DataLoader
import os
import SimpleITK as sitk
from zipfile import ZipFile
from zipfile import BadZipFile
import dask.dataframe as dd
import os
import multiprocessing as mp
import functools
from functools import partial
import preprocessing.Standardize as Standardize

df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
path=df['adc'].to_numpy()[50]
image=sitk.ReadImage(path)
imageB=Standardize.removeOutliersBiasFieldCorrect(path)
outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", "f")
sitkUtils.PushVolumeToSlicer(imageB, outputVolumeNode)

<MRMLCore.vtkMRMLScalarVolumeNode(0xe383c20) at 0x7fb96b0cf9a0>

In [14]:
from os.path import isdir,join,exists,split,dirname,basename
seriesString= "t2w"
trainedModelsBasicPath='/home/sliceruser/data/preprocess/standarizationModels'
trainedModel=join(trainedModelsBasicPath,'trained_model'+seriesString+'.npz')
f = np.load(trainedModel, allow_pickle=True)
Model = f['trainedModel'].all()
meanLandmarks = Model['meanLandmarks']
meanLandmarks

array([ 1.        , 61.21375865, 64.7723805 , 67.79212988, 70.38931772,
       72.68893844, 75.07817142, 77.49362605, 80.70655573, 86.00811824,
       99.        ])

In [15]:
imagC= Standardize.transform(imageB,meanLandmarks=meanLandmarks)
outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", "g")
sitkUtils.PushVolumeToSlicer(imagC, outputVolumeNode)

<MRMLCore.vtkMRMLScalarVolumeNode(0xe7d4550) at 0x7fbb485f9be0>